### Importation des librairies

In [1]:
# Manipulation des données
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import recall_score, make_scorer

### Importation des données

In [2]:
df_final = pd.read_csv("Data/df_knn.csv", sep = ',')

In [3]:
df_final.head()

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,1,171.81,104.86,104.95,4.52,2.89,112.83
1,1,171.46,103.36,103.66,3.77,2.99,113.09
2,1,172.69,104.48,103.50,4.40,2.94,113.16
3,1,171.36,103.91,103.94,3.62,3.01,113.51
4,1,171.73,104.28,103.46,4.04,3.48,112.54


Nous allons entrainer le modèle sur l'ensemble des données à notre disposition, soit les 1500 billets.

In [4]:
# On sépare les variables explicatives de la variable cible
X = df_final.drop(columns="is_genuine")
y = df_final["is_genuine"]

On détermine le nombre de k voisins qui donne la meilleure spécificité

In [5]:
param_grid = {'n_neighbors': range(2,16)}
specificity_score = make_scorer(recall_score, pos_label=0)

# Paramètre que l'on veut tester
param_grid = {'n_neighbors': range(2,16)}

knn_2 = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid,     
    cv=5,           
    scoring=specificity_score)

knn_2.fit(X, y)
              
best_k = knn_2.best_params_

# On affiche le meilleur paramètre
print(f"Nombre de voisins k le plus adapté: {best_k}")

Nombre de voisins k le plus adapté: {'n_neighbors': 2}


On relance l'algorithme avec k=2

In [6]:
knn = KNeighborsClassifier(2)
knn.fit(X, y)

KNeighborsClassifier(n_neighbors=2)

On crée une fonction qui prend en entrée un fichier contenant des dimensions de billets et qui renvoie ce fichier avec les résultats des prédictions et les probabilités correspondantes.

In [7]:
def detecteur(fichier):
    if  fichier.shape[1] != 7: # On vérifie la taille du fichier en entrée
        print("La taille du fichier n'est pas valide !")
    else:
        # On exclu la variable 'id' du jeu de données nécessaire à la prédiction
        X = fichier.drop(columns="id")
        
        # On prédit la nature des billets
        fichier["résultat"] = knn.predict(X)
        fichier["proba_False"] = knn.predict_proba(X)[:,0]
        fichier["proba_True"] = knn.predict_proba(X)[:,1]
        
        # On remplace 0 par False et 1 par True
        fichier["résultat"] = fichier["résultat"].apply(lambda x: False if x == 0 else True)
        
        return fichier

In [8]:
data = pd.read_csv("Data/billets_test.csv", sep = ',')

In [9]:
data

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


In [10]:
detecteur(data)

,diagonal,height_left,height_right,margin_low,margin_up,length,id,résultat,proba_False,proba_True
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,True,0.0,1.0
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,False,1.0,0.0
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,True,0.0,1.0
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,False,1.0,0.0
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,False,1.0,0.0
